# Init

## Install Dependencies

In [1]:
!pip install --upgrade openpyxl xlrd pandas

Requirement already up-to-date: openpyxl in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (2.5.3)
Requirement already up-to-date: xlrd in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (1.1.0)
Requirement already up-to-date: pandas in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (0.23.0)
Requirement not upgraded as not directly required: et-xmlfile in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (from openpyxl) (1.0.1)
Requirement not upgraded as not directly required: jdcal in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (from openpyxl) (1.4)
Requirement not upgraded as not directly required: pytz>=2011k in /Users/devvyn-70504/.local/share/virtualenvs/notebook-qwFIhkl_/lib/python3.6/site-packages (from pandas) (2018.4)
Requirement not upgraded as not directly re

## Import Python Modules

In [2]:
import pandas, IPython.core.display

## Open Workbook File

In [3]:
src = pandas.ExcelFile('2017 CAM data from iPads.xlsx')
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## Select Sheets

In [4]:
search_strings = ['Erl', 'Tyler']
sheets_to_process = {
    sheet_name.split(' ')[-1]: src.parse(sheet_name)
    for sheet_name in src.sheet_names
    if any(
        (
            (pattern in sheet_name) for pattern in search_strings
        )
    )
}
sheets_to_process.keys()

dict_keys(['Erl', 'Tyler'])

## Define Aphids and Natural Enemies

In [5]:
EGA = 'English grain'
BCO = 'Bird-cherry oat'
greenbug = 'green bug'
aphid_name_list = (EGA, BCO, greenbug)
aphid_name = {f'a{n + 1}': name for n, name in enumerate(aphid_name_list)}
aphid_name

{'a1': 'English grain', 'a2': 'Bird-cherry oat', 'a3': 'green bug'}

In [6]:
natural_enemy_name_list = [
    'lady_beetle_7_adult',
    'lady_beetle_7_larva',
    'lady_beetle_13_adult',
    'lady_beetle_13_larva',
    'green_lacewing_larva',
    'damsel_bug_adult',
    'minute_pirate_bug_adult',
    'aphid_mummy_brown',
    'aphid_mummy_black',
]
natural_enemy_name = {f'e{n + 1}': name for n, name in enumerate(natural_enemy_name_list)}
natural_enemy_name

{'e1': 'lady_beetle_7_adult',
 'e2': 'lady_beetle_7_larva',
 'e3': 'lady_beetle_13_adult',
 'e4': 'lady_beetle_13_larva',
 'e5': 'green_lacewing_larva',
 'e6': 'damsel_bug_adult',
 'e7': 'minute_pirate_bug_adult',
 'e8': 'aphid_mummy_brown',
 'e9': 'aphid_mummy_black'}

# Cleanup

## Normalize Column Names

In [7]:
sheets_needing_column_rename = (
    (sheet_name, sheet)
    for sheet_name, sheet
    in sheets_to_process.items()
    if any(
        (str(column).find(' ')
         for column in sheet.columns)
    )
)
sheets_to_process = {
    sheet_name: sheet.rename(
        mapper=lambda x: str(x).split(' ')[0],
        axis='columns'
    )
    for sheet_name, sheet in sheets_needing_column_rename
}

## Concatenate Sheets

In [8]:
concat_df = pandas.concat(
    sheets_to_process,
    names=['worksheet_name', 'index'],
    sort=True
)

## Rename Columns

In [9]:
names = [
     'scope',       'group',         'variable',          'source_field']
columns = (
    ('field',       '',              'client',             'fields__client__displayText'),
    ('field',       'index',         'crop',               'fields__crop'),
    ('field',       'index',         'name',               'fields__name'),
    ('field',       'index',         'description',        'fields__desc'),
    ('field',       '',              'date',               'fields__date'),
    ('set',         'index',         'date',               'fields__oSets__date'),
    ('set',         '',              'description',        'fields__oSets__desc'),
    ('set',         'category',      'zadoks',             'fields__oSets__growthStage'),
    ('set',         'category',      'observer',           'fields__oSets__obsName'),
    ('set',         'calculated',    'complete',           'fields__oSets__completeSets'),
    ('set',         'calculated',    'total',              'fields__oSets__totalSets'),
    ('point',       'index',         'id',                 'fields__oSets__oPoints__id'),
    ('observation', 'index',         'id',                 'fields__oSets__oPoints__observations__id'),
    ('observation', 'boolean',       'complete',           'fields__oSets__oPoints__observations__complete'),
    ('observation', 'boolean',       'disabled',           'fields__oSets__oPoints__observations__disabled'),
    ('observation', 'calculated',    'natural_enemy_total','fields__oSets__oPoints__observations__enum'),
    ('observation', 'calculated',    'aphid_total',        'fields__oSets__oPoints__observations__anum'),
    ('observation', 'aphid',         'a1',                 'fields__oSets__oPoints__observations__a1__number'),
    ('observation', 'aphid',         'a2',                 'fields__oSets__oPoints__observations__a2__number'),
    ('observation', 'aphid',         'a3',                 'fields__oSets__oPoints__observations__a3__number'),
    ('observation', 'natural_enemy', 'name',               'fields__oSets__oPoints__observations__|'),
    ('observation', 'natural_enemy', 'number',             'fields__oSets__oPoints__observations__|__number'),
)
df = concat_df.reindex(
    columns=pandas.MultiIndex.from_tuples(columns, names=names),
    level='source_field')
df.head()

scope                                      field                            \
group                                                   index                
variable                                  client         crop         name   
source_field         fields__client__displayText fields__crop fields__name   
worksheet_name index                                                         
Erl            0                             NaN          NaN          NaN   
               1                             NaN          NaN          NaN   
               2                             NaN          NaN          NaN   
               3                             NaN          NaN          NaN   
               4                             NaN          NaN          NaN   

scope                                                               set  \
group                                                             index   
variable              description         date                     date   
source_field         fields__desc fields__date      fields__oSets__date   
worksheet_name index                                                      
Erl            0              NaN          NaN  2017-08-02T13:12:09.542   
               1              NaN          NaN                      NaN   
               2              NaN          NaN                      NaN   
               3              NaN          NaN                      NaN   
               4              NaN          NaN                      NaN   

scope                                                                \
group                                                      category   
variable                     description                     zadoks   
source_field         fields__oSets__desc fields__oSets__growthStage   
worksheet_name index                                                  
Erl            0                     NaN                        7.0   
               1                     NaN                        NaN   
               2                     NaN                        NaN   
               3                     NaN                        NaN   
               4                     NaN                        NaN   

scope                                                                    \
group                                                        calculated   
variable                           observer                    complete   
source_field         fields__oSets__obsName fields__oSets__completeSets   
worksheet_name index                                                      
Erl            0                      Tyler                         0.0   
               1                        NaN                         NaN   
               2                        NaN                         NaN   
               3                        NaN                         NaN   
               4                        NaN                         NaN   

scope                                      ...                        \
group                                      ...                         
variable                                   ...                         
source_field                               ...                         
worksheet_name index                       ...                         
Erl            0                           ...                         
               1                           ...                         
               2                           ...                         
               3                           ...                         
               4                           ...                         

scope                                             observation  \
group                                                   index   
variable                                                   id   
source_field         fields__oSets__oPoints__observations__id   
worksheet_name index     

### Rename Aphids (a1, a2, …)

In [10]:
df = df.rename(columns=aphid_name, level='variable')
df.head()

scope                                      field                            \
group                                                   index                
variable                                  client         crop         name   
source_field         fields__client__displayText fields__crop fields__name   
worksheet_name index                                                         
Erl            0                             NaN          NaN          NaN   
               1                             NaN          NaN          NaN   
               2                             NaN          NaN          NaN   
               3                             NaN          NaN          NaN   
               4                             NaN          NaN          NaN   

scope                                                               set  \
group                                                             index   
variable              description         date                     date   
source_field         fields__desc fields__date      fields__oSets__date   
worksheet_name index                                                      
Erl            0              NaN          NaN  2017-08-02T13:12:09.542   
               1              NaN          NaN                      NaN   
               2              NaN          NaN                      NaN   
               3              NaN          NaN                      NaN   
               4              NaN          NaN                      NaN   

scope                                                                \
group                                                      category   
variable                     description                     zadoks   
source_field         fields__oSets__desc fields__oSets__growthStage   
worksheet_name index                                                  
Erl            0                     NaN                        7.0   
               1                     NaN                        NaN   
               2                     NaN                        NaN   
               3                     NaN                        NaN   
               4                     NaN                        NaN   

scope                                                                    \
group                                                        calculated   
variable                           observer                    complete   
source_field         fields__oSets__obsName fields__oSets__completeSets   
worksheet_name index                                                      
Erl            0                      Tyler                         0.0   
               1                        NaN                         NaN   
               2                        NaN                         NaN   
               3                        NaN                         NaN   
               4                        NaN                         NaN   

scope                                      ...                        \
group                                      ...                         
variable                                   ...                         
source_field                               ...                         
worksheet_name index                       ...                         
Erl            0                           ...                         
               1                           ...                         
               2                           ...                         
               3                           ...                         
               4                           ...                         

scope                                             observation  \
group                                                   index   
variable                                                   id   
source_field         fields__oSets__oPoints__observations__id   
worksheet_name index     

## Normalize Text Labels

In [11]:
translation_map = str.maketrans(' -,', '.' * 3)
def string_clean(s):
    if isinstance(s, str):
        return s.rstrip().title().translate(translation_map)
    return s


text_columns = df.loc[:, pandas.IndexSlice['field', :, 'name']].columns
df[text_columns] = df[text_columns].applymap(string_clean)
df[text_columns].dropna(how='all').head()

scope                             field
group                             index
variable                           name
source_field               fields__name
worksheet_name index                   
Erl            70     Llewellyn.Wheat.1
               140            Llewellyn
               210           Sef.Barley
               350             Sef.Oats
               490         Kernen.Wheat

## Assign Column Data Types

In [12]:
cat_cols = df.loc[:, pandas.IndexSlice[:, 'category']].columns
df[cat_cols] = df[cat_cols].astype('category')
# for column in cat_cols:
#     df[column] = df[column].astype('category')

text_cols = df.loc[:, pandas.IndexSlice[:, :, ['name', 'description']]].columns
display(text_cols.values)
df[text_cols] = df[text_cols].astype('object')
# for column in text_cols:
#     df[column] = df[column].astype('object')

boolean_cols = df.loc[:, pandas.IndexSlice[:, 'boolean']].columns
df[boolean_cols] = df[boolean_cols].fillna(False).astype('bool')
# for column in boolean_cols:
#     df[column] = df[column].astype('bool')

df.info()

array([('field', 'index', 'name', 'fields__name'),
       ('field', 'index', 'description', 'fields__desc'),
       ('set', '', 'description', 'fields__oSets__desc'),
       ('observation', 'natural_enemy', 'name', 'fields__oSets__oPoints__observations__|')],
      dtype=object)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4690 entries, (Erl, 0) to (Tyler, 3779)
Data columns (total 22 columns):
(field, , client, fields__client__displayText)                                                21 non-null object
(field, index, crop, fields__crop)                                                            21 non-null object
(field, index, name, fields__name)                                                            21 non-null object
(field, index, description, fields__desc)                                                     15 non-null object
(field, , date, fields__date)                                                                 21 non-null object
(set, index, date, fields__oSets__date)                                                       54 non-null object
(set, , description, fields__oSets__desc)                                                     0 non-null object
(set, category, zadoks, fields__oSets__growthStage)                                    

## Convert Datetime

In [13]:
date_column_mask, date_column_slice = df.columns.get_loc_level('date', level='variable')

In [14]:
df.loc[:, date_column_mask] = df.loc[:, date_column_mask].apply(pandas.to_datetime)
df.loc[:, date_column_mask].dropna(how='all').head()

scope                                  field                     set
group                                                          index
variable                                date                    date
source_field                    fields__date     fields__oSets__date
worksheet_name index                                                
Erl            0                         NaT 2017-08-02 13:12:09.542
               70    2017-08-09 09:24:11.845 2017-08-09 09:25:11.710
               140   2017-08-09 10:01:29.326 2017-08-09 10:06:25.480
               210   2017-08-09 11:16:15.922 2017-08-09 11:21:01.555
               350   2017-08-09 11:17:15.791 2017-08-09 11:37:20.862

## Apply Index Columns

In [15]:
# icols = df.columns.get_loc_level('index', level='group', drop_level=False)[1].values.tolist()
icols = df.xs('index', level='group', axis='columns', drop_level=False).columns.values.tolist()
icols

[('field', 'index', 'crop', 'fields__crop'),
 ('field', 'index', 'name', 'fields__name'),
 ('field', 'index', 'description', 'fields__desc'),
 ('set', 'index', 'date', 'fields__oSets__date'),
 ('point', 'index', 'id', 'fields__oSets__oPoints__id'),
 ('observation', 'index', 'id', 'fields__oSets__oPoints__observations__id')]

In [16]:
df[icols] = df[icols].ffill()

df = df.set_index(icols, append=True)
df.head(7)

scope                                                                                                                                                                                                                                                                                                field  \
group                                                                                                                                                                                                                                                                                                        
variable                                                                                                                                                                                                                                                                                            client   
source_field                                                                                                                                                                                                                                                                   fields__client__displayText   
worksheet_name index (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                               
Erl            0     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            0.0                                                                        NaN   
               1     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            1.0                                                                        NaN   
               2     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            2.0                                                                        NaN   
               3     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            3.0                                                                        NaN   
               4     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            4.0                                                                        NaN   
               5     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               6     NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   

scope                                                                                                                                                                                                                                                                                        \
group                                                                                   

## Unstack Natural Enemies

In [17]:
ne = (
    df
    .xs('natural_enemy', level='group', axis='columns', drop_level=False)
    .xs(5.0, level=7, drop_level=False)
    .set_index(('observation', 'natural_enemy', 'name', 'fields__oSets__oPoints__observations__|'), append=True)
    .unstack()
#     .sum(level=[2, 3, 4, 5, 6, 7])
    .sum(level=[0, 2, 3, 4, 5, 6, 7])
#     .sort_index(level=list(range(6)))
    .sort_index(level=list(range(7)))
#     .iloc[::9]
)
ne.head(15)

scope                                                                                                                                                                                                                                                                                                        observation  \
group                                                                                                                                                                                                                                                                                                      natural_enemy   
variable                                                                                                                                                                                                                                                                                                          number   
source_field                                                                                                                                                                                                                                                             fields__oSets__oPoints__observations__|__number   
('observation', 'natural_enemy', 'name', 'fields__oSets__oPoints__observations__|')                                                                                                                                                                                                                                   e1   
worksheet_name (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                                                   
Erl            CROPS.BARLEY                       Sef.Barley                         Next to SEF Wheat and near faba beans     2017-08-09 11:21:01.555                 0.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       1.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       2.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       3.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       4.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       5.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       6.0                                      

### Rename Natural Enemies (e1, e2, …)

In [18]:
levels = [(scope, group, ne, source_field) for scope, group, variable, source_field, ne in ne.columns.values]
ne.columns = pandas.MultiIndex.from_tuples(levels, names=ne.columns.names[:-1])
ne = ne.rename(columns=natural_enemy_name, level='variable')
ne.head()

scope                                                                                                                                                                                                                                                                                                        observation  \
group                                                                                                                                                                                                                                                                                                      natural_enemy   
variable                                                                                                                                                                                                                                                                                             lady_beetle_7_adult   
source_field                                                                                                                                                                                                                                                             fields__oSets__oPoints__observations__|__number   
worksheet_name (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                                                   
Erl            CROPS.BARLEY                       Sef.Barley                         Next to SEF Wheat and near faba beans     2017-08-09 11:21:01.555                 0.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       1.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       2.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       3.0                                            5.0                                                                                            0.0   
                                                                                                                                                                       4.0                                            5.0                                                                                            0.0   

scope                                                                                                                                                                                                                                                                                                                     \
group                                                                                                                                                                                                                                                                                                                      
variable                                                                                                                                                                                                       

### Rejoin Unstacked Natural Enemies

In [19]:
df2 = (
    df
    .reset_index('index', drop=True)
    .sort_index(level=list(range(6)))
)
df2 = df2.loc[~df2.index.duplicated(), pandas.IndexSlice[:, set(df2.columns.unique(level='group')) - {'natural_enemy'}]]
df2.head(7)

scope                                                                                                                                                                                                                                                                                          field  \
group                                                                                                                                                                                                                                                                                                  
variable                                                                                                                                                                                                                                                                                      client   
source_field                                                                                                                                                                                                                                                             fields__client__displayText   
worksheet_name (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                               
Erl            NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            0.0                                                                        NaN   
                                                                                                                                                                                                                      1.0                                                                        NaN   
                                                                                                                                                                                                                      2.0                                                                        NaN   
                                                                                                                                                                                                                      3.0                                                                        NaN   
                                                                                                                                                                                                                      4.0                                                                        NaN   
                                                                                                                                                                                                                      5.0                                                                        NaN   
                                                                                                                                                                       1.0                                            0.0                                                                        NaN   

scope                                                                                                                                                                                                                                                                                  \
group                                                                                                                                                                 

In [20]:
columns = list(df2.columns.values) + list(ne.columns.values)
df3 = (
    pandas.concat([df2, ne], axis='columns')[columns]
    .reset_index(level='worksheet_name')
    .sort_index(level=list(range(6)))
)
pandas.options.display.max_columns=55
df3.head(7)

scope                                                                                                                                                                                                                                                     worksheet_name  \
group                                                                                                                                                                                                                                                                      
variable                                                                                                                                                                                                                                                                   
source_field                                                                                                                                                                                                                                                               
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                  
NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            0.0                                                           Erl   
                                                                                                                                                                                                       1.0                                                           Erl   
                                                                                                                                                                                                       2.0                                                           Erl   
                                                                                                                                                                                                       3.0                                                           Erl   
                                                                                                                                                                                                       4.0                                                           Erl   
                                                                                                                                                                                                       5.0                                                           Erl   
                                                                                                                                                        1.0                                            0.0                                                           Erl   

scope                                                                                                                                                                                                                                                                           field  \
group                                                                                                                                                                                                                                                                                   
variable                                                                                                                                                                                                                     

## Fill Columns

In [21]:
field_set_columns = df3.loc[
    :,
    pandas.IndexSlice[
        ['field', 'set'],
        set(df3.columns.unique(level='group')) - {'calculated'}
    ]
].columns
field_set_columns.values

array([('field', '', 'client', 'fields__client__displayText'),
       ('field', '', 'date', 'fields__date'),
       ('set', '', 'description', 'fields__oSets__desc'),
       ('set', 'category', 'zadoks', 'fields__oSets__growthStage'),
       ('set', 'category', 'observer', 'fields__oSets__obsName')],
      dtype=object)

In [22]:
df3[field_set_columns] = df3[field_set_columns].fillna(method='ffill', downcast='infer')
df3[field_set_columns].head()

scope                                                                                                                                                                                                                                                                           field  \
group                                                                                                                                                                                                                                                                                   
variable                                                                                                                                                                                                                                                                       client   
source_field                                                                                                                                                                                                                                              fields__client__displayText   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                               
NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            0.0                                                                        NaN   
                                                                                                                                                                                                       1.0                                                                        NaN   
                                                                                                                                                                                                       2.0                                                                        NaN   
                                                                                                                                                                                                       3.0                                                                        NaN   
                                                                                                                                                                                                       4.0                                                                        NaN   

scope                                                                                                                                                                                                                                                                   \
group                                                                                                                                                                                                                                                                    
variable                                                                                                                                                                                                                                                          date   
source_field                                                                                                                                                                                                                                              fields__date   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, 

In [23]:
aphid_columns = df3.loc[:, pandas.IndexSlice[:, 'aphid']].columns
df3[aphid_columns] = df3.loc[pandas.IndexSlice[:, :, :, :, :, 0:4], aphid_columns].fillna(0)
df3[aphid_columns].head(7)

scope                                                                                                                                                                                                                                                                                          observation  \
group                                                                                                                                                                                                                                                                                                aphid   
variable                                                                                                                                                                                                                                                                                     English grain   
source_field                                                                                                                                                                                                                                              fields__oSets__oPoints__observations__a1__number   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                                                    
NaN                                NaN                                NaN                                       2017-08-02 13:12:09.542                 0.0                                            0.0                                                                                             0.0   
                                                                                                                                                                                                       1.0                                                                                             0.0   
                                                                                                                                                                                                       2.0                                                                                             0.0   
                                                                                                                                                                                                       3.0                                                                                             0.0   
                                                                                                                                                                                                       4.0                                                                                             0.0   
                                                                                                                                                                                                       5.0                                                                                             NaN   
                                                                                                                                                        1.0                                            0.0                                                                                             0.0   

scope                                                                                                                                                                                                                                                                                                       \
group                                                                    

In [24]:
all_observations = df3.copy()

# Generate Sums

In [25]:
sums = {
    ' '.join((df3.index.names[-1 - depth][x] for x in (0, 2))): 
    df3.sum(level=list(range(df3.index.nlevels - depth)), min_count=1)
    for depth in range(1, df3.index.nlevels)
}

In [26]:
sums.keys()

dict_keys(['point id', 'set date', 'field description', 'field name', 'field crop'])

In [27]:
sums['field crop']

scope                                                      set  \
group                                               calculated   
variable                                              complete   
source_field                       fields__oSets__completeSets   
(field, index, crop, fields__crop)                               
CROPS.BARLEY                                              17.0   
CROPS.OATS                                                 8.0   
CROPS.WHEAT                                               39.0   

scope                                                        \
group                                                         
variable                                              total   
source_field                       fields__oSets__totalSets   
(field, index, crop, fields__crop)                            
CROPS.BARLEY                                           17.0   
CROPS.OATS                                              8.0   
CROPS.WHEAT                                            41.0   

scope                                                                 observation  \
group                                                                     boolean   
variable                                                                 complete   
source_field                       fields__oSets__oPoints__observations__complete   
(field, index, crop, fields__crop)                                                  
CROPS.BARLEY                                                                510.0   
CROPS.OATS                                                                  240.0   
CROPS.WHEAT                                                                1182.0   

scope                                                                              \
group                                                                               
variable                                                                 disabled   
source_field                       fields__oSets__oPoints__observations__disabled   
(field, index, crop, fields__crop)                                                  
CROPS.BARLEY                                                                  0.0   
CROPS.OATS                                                                    0.0   
CROPS.WHEAT                                                                   0.0   

scope                                                                          \
group                                                              calculated   
variable                                                  natural_enemy_total   
source_field                       fields__oSets__oPoints__observations__enum   
(field, index, crop, fields__crop)                                              
CROPS.BARLEY                                                              6.0   
CROPS.OATS                                                                1.0   
CROPS.WHEAT                                                             397.0   

scope                                                                          \
group                                                                           
variable                                                          aphid_total   
source_field                       fields__oSets__oPoints__observations__anum   
(field, index, crop, fields__crop)                                              
CROPS.BARLEY                                                            181.0   
CROPS.OATS                                                               98.0   
CROPS.WHEAT                                                            3962.0   

scope                                                                                \
group                                                                         aphid   
variable                                                              English grain   
source_field                       fields__oS

In [28]:
sums['field name'].head()

scope                                                                                         set  \
group                                                                                  calculated   
variable                                                                                 complete   
source_field                                                          fields__oSets__completeSets   
(field, index, crop, fields__crop) (field, index, name, fields__name)                               
CROPS.BARLEY                       Alvena.Barley                                              3.0   
                                   Llewelyn.Barley                                            6.0   
                                   Sef.Barley                                                 8.0   
CROPS.OATS                         Sef.Oats                                                   8.0   
CROPS.WHEAT                        Alvena.Wheat                                               4.0   

scope                                                                                           \
group                                                                                            
variable                                                                                 total   
source_field                                                          fields__oSets__totalSets   
(field, index, crop, fields__crop) (field, index, name, fields__name)                            
CROPS.BARLEY                       Alvena.Barley                                           3.0   
                                   Llewelyn.Barley                                         6.0   
                                   Sef.Barley                                              8.0   
CROPS.OATS                         Sef.Oats                                                8.0   
CROPS.WHEAT                        Alvena.Wheat                                            4.0   

scope                                                                                                    observation  \
group                                                                                                        boolean   
variable                                                                                                    complete   
source_field                                                          fields__oSets__oPoints__observations__complete   
(field, index, crop, fields__crop) (field, index, name, fields__name)                                                  
CROPS.BARLEY                       Alvena.Barley                                                                90.0   
                                   Llewelyn.Barley                                                             180.0   
                                   Sef.Barley                                                                  240.0   
CROPS.OATS                         Sef.Oats                                                                    240.0   
CROPS.WHEAT                        Alvena.Wheat                                                                120.0   

scope                                                                                                                 \
group                                                                                                                  
variable                                                                                                    disabled   
source_field                                                          fields__oSets__oPoints__observations__disabled   
(field, index, crop, fields__crop) (field, index, name, fields__name)                                                  
CROPS.BARLEY                       Alvena.Barley                                                                 0.0   
                                   Llewelyn.Barley                                     

In [29]:
sums['field description'].head()

scope                                                                                                                                   set  \
group                                                                                                                            calculated   
variable                                                                                                                           complete   
source_field                                                                                                    fields__oSets__completeSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc)                               
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola                                       3.0   
                                   Llewelyn.Barley                    Next to SEF Peas, near the canola                                 6.0   
                                   Sef.Barley                         Next to SEF Wheat and near faba beans                             2.0   
                                                                      Right next to SEF wheat                                           6.0   
CROPS.OATS                         Sef.Oats                           Next to SEF Peas, near the canola                                 6.0   

scope                                                                                                                                     \
group                                                                                                                                      
variable                                                                                                                           total   
source_field                                                                                                    fields__oSets__totalSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc)                            
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola                                    3.0   
                                   Llewelyn.Barley                    Next to SEF Peas, near the canola                              6.0   
                                   Sef.Barley                         Next to SEF Wheat and near faba beans                          2.0   
                                                                      Right next to SEF wheat                                        6.0   
CROPS.OATS                         Sef.Oats                           Next to SEF Peas, near the canola                              6.0   

scope                                                                                                                                              observation  \
group                                                                                                                                                  boolean   
variable                                                                                                                                              complete   
source_field                                                                                                    fields__oSets__oPoints__observations__complete   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc)                                                  
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola                                                         90.0   
                                   Llewelyn.Barley                    Next to SEF Peas, near the canola                                                  180.0   
                                  

In [30]:
sums['set date'].head()

scope                                                                                                                                                                           set  \
group                                                                                                                                                                    calculated   
variable                                                                                                                                                                   complete   
source_field                                                                                                                                            fields__oSets__completeSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date)                               
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola               2017-07-19 10:22:29.528                                         1.0   
                                                                                                                2017-07-28 11:07:21.041                                         1.0   
                                                                                                                2017-08-02 15:06:10.327                                         1.0   
                                   Llewelyn.Barley                    Next to SEF Peas, near the canola         2017-07-18 14:49:08.831                                         1.0   
                                                                                                                2017-07-28 14:34:36.584                                         1.0   

scope                                                                                                                                                                             \
group                                                                                                                                                                              
variable                                                                                                                                                                   total   
source_field                                                                                                                                            fields__oSets__totalSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date)                            
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola               2017-07-19 10:22:29.528                                      1.0   
                                                                                                                2017-07-28 11:07:21.041                                      1.0   
                                                                                                                2017-08-02 15:06:10.327                                      1.0   
                                   Llewelyn.Barley                    Next to SEF Peas, near the canola         2017-07-18 14:49:08.831                                      1.0   
                                                                                                                2017-07-28 14:34:36.584                                      1.0   

scope                                                                                                                                                                                      observation  \
group                                                                                                                                                                 

In [31]:
sums['point id'].head()

scope                                                                                                                                                                                                                          set  \
group                                                                                                                                                                                                                   calculated   
variable                                                                                                                                                                                                                  complete   
source_field                                                                                                                                                                                           fields__oSets__completeSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id)                               
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola               2017-07-19 10:22:29.528                 0.0                                                                    1.0   
                                                                                                                                                        1.0                                                                    NaN   
                                                                                                                                                        2.0                                                                    NaN   
                                                                                                                                                        3.0                                                                    NaN   
                                                                                                                                                        4.0                                                                    NaN   

scope                                                                                                                                                                                                                            \
group                                                                                                                                                                                                                             
variable                                                                                                                                                                                                                  total   
source_field                                                                                                                                                                                           fields__oSets__totalSets   
(field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, description, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id)                            
CROPS.BARLEY                       Alvena.Barley                      Right next to Alvena Canola               2017-07-19 10:22:29.528                 0.0                                                                 1.0   
                                                                                                                                                        1.0                                                                 NaN   
                                                                                                              

# Save File

In [32]:
frames = [all_observations] + list(sums.values())
for frame in frames:
    frame.reset_index(inplace=True)
    frame.columns = (
        frame.columns
        .droplevel(level=['group', 'source_field'])
        .map(lambda x: '_'.join(y.replace(' ', '_') for y in x if y))
    )

In [33]:
filename = 'CAM-2017.xlsx'
with pandas.ExcelWriter(filename) as file_writer:
    args = dict(
            excel_writer=file_writer,
            freeze_panes=(1, 1),
            index_label='row'
    )
    all_observations.to_excel(
        **args,
        sheet_name='all observations',
    )
    for name, frame in sums.items():
        frame.to_excel(
            **args,
            sheet_name=f'sum by {name}',
        )
    file_writer.save()